In [1]:
%%html
<link rel="stylesheet" type="text/css" href="theme/sixty_north.css">

In [2]:
%run unittest_magics.py

# `unittest`
## Testing with the standard library

[docs.python.org/3/library/unittest.html](http://docs.python.org/3/library/unittest.html)

# Basic features

* Test automation
* Sharing of setup and shutdown code for tests
* Aggregation of tests into collections
* Independence of the tests from the reporting framework.

# `unittest.TestCase`
## The basic unit of test organization

* provides assertion methods
* test and class fixtures
* related test methods



**To create a test**, subclass from `unittest.TestCase` and implement test methods.

# The simplest `TestCase` example

In [3]:
%%unittest_run

# This TestCase does nothing
class Tests(unittest.TestCase):
    pass

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 0


Not terribly exciting!

# Test methods start with "test"

Actual testing occurs in `TestCase` methods that start with "test".

In [4]:
%%unittest_run

class Tests(unittest.TestCase):
    # test_ indicates a test to be run. This one does nothing.
    def test_empty(self):
        pass
    
    # So does just "test", but it's less Pythonic
    def testNothing(self):
        pass
    
    # This test isn't found because it doesn't start with test
    def empty_test(self):
        pass

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 2


Still not very interesting, but we see that a test has been run!

# Assertions
## Expressions of expectations

`TestCase` provides assertion methods that determine if tests pass or fail. 

In [5]:
%%unittest_run

class Tests(unittest.TestCase):
    # This test passes trivially
    def test_always_passes(self):
        self.assertTrue(True)
        
    # This test fails trivially
    def test_always_fails(self):
        self.assertFalse(True)

FAILURES: 1 ERRORS: 0 SKIPPED: 0 TOTAL: 2
f: test_always_fails (__main__.Tests)


# Common assertions

| method name                 | checks               |
|-----------------------------|----------------------|
| `assertEqual(a, b)`         | `a == b`             | 
| `assertNotEqual(a, b)`      | `a != b`             | 
| `assertTrue(x)`	          | `bool(x) is True`    | 
| `assertFalse(x)`            | `bool(x) is False`   | 
| `assertIs(a, b)`		      | `a is b`             | 
| `assertIsNot(a, b)`		  | `a is not b`         |

| method name                | checks                 |
|----------------------------|------------------------|
| `assertIsNone(x)`	         | `x is None`            |
| `assertIsNotNone(x)`		 | `x is not None`        |
| `assertIn(a, b)`		     | `a in b`               |
| `assertNotIn(a, b)`        | `a not in b`           |
|`assertIsInstance(a, b)`	 | `isinstance(a, b)`     |
|`assertNotIsInstance(a, b)` | `not isinstance(a, b)` |

See full details in the [`TestCase` documentation](https://docs.python.org/3/library/unittest.html#unittest.TestCase)

# Testing for exceptions
`assertRaises` is a *context-manager* that checks for an exception.

In [6]:
%%unittest_run

class Tests(unittest.TestCase):
    # Verify that an IndexError is thrown in the test
    def test_index_error(self): 
        with self.assertRaises(IndexError):
            raise IndexError()
            
    def test_value_error(self):
        # Verify that a ValueError is thrown in the test
        with self.assertRaises(ValueError):
            pass

FAILURES: 1 ERRORS: 0 SKIPPED: 0 TOTAL: 2
f: test_value_error (__main__.Tests)


# Fixtures
## Test set-up and clean-up

Code run at various times in the test run:

| name                     | type            | when                         |
|--------------------------|-----------------|------------------------------|
| `TestCase.setUp`         | instance method | before each test method      |
| `TestCase.tearDown`      | instance method | after each test method       |
| `TestCase.setUpClass`    | class method    | before any methods run       |
| `TestCase.tearDownClass` | class method    | after all methods run        |
| `setUpModule`            | global function | before any methods in module |
| `tearDownModule`         | global function | after all methods in module  |

# What are fixtures for?
They ensure a known state for tests and clean up resources.

* Create/clear database tables
* Configure instance members
* Create/delete temporary files

In [7]:
%%unittest_run -n

class Tests(unittest.TestCase):
    # This is run once before any of the tests in the TestCase
    @classmethod
    def setUpClass(cls): print('set up class')
        
    # This is run once after all of the tests in the TestCase have run
    @classmethod
    def tearDownClass(cls): print('tear down class')
        
    # This is run before *each* test in the TestCase
    def setUp(self): print('- set up')
        
    # This is run after *each* test in the TestCase
    def tearDown(self): print('- tear down')

In [8]:
%%unittest_run

    # A few demo tests
    def test_one(self): print('-- test one')
    def test_two(self): print('-- test two')

set up class
- set up
-- test one
- tear down
- set up
-- test two
- tear down
tear down class
FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 2


# Configuring member attributes

`setUp()` is often used to assign to attributes used in tests.

In [9]:
%%unittest_run

class Tests(unittest.TestCase):
    def setUp(self):
        # self.data will be set to this value before each test in the TestCase
        self.data = {'a': 2} 
        
    def test_foo(self):
        # We can test the value of self.data
        self.assertEqual(self.data['a'], 2)
        
        # Modifications to self.data will be reset by setUp()
        self.data['a'] = 42
        
    def test_bar(self):
        self.assertEqual(self.data['a'], 2)
        self.data['a'] = 1337

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 2


# `unittest.main()`
*Convenience function for running from the command line*

Finds all tests in a module and executes them.

```python
# unittest_main.py

import unittest

class Tests(unittest.TestCase):
    # This fails trivially
    def test_foo(self):
        self.assertEqual(1, 2)

# This the idiomatic test for "am I the top-level 
# program" in Python
if __name__ == '__main__':
    # If this is the top-level program, this scans 
    # for all TestCases and runs them.
    unittest.main()
 ```

In [10]:
%run -e examples/unittest_main.py

F
FAIL: test_foo (__main__.Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/abingham/repos/python-testing-presentation/slides/examples/unittest_main.py", line 7, in test_foo
    self.assertEqual(1, 2)
AssertionError: 1 != 2

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


# Skipping tests with decorators
*Use the decorators `skip()`, `skipIf()`, and `skipUnless()` to skip tests*

In [11]:
%%unittest_run

# This controls which tests run
cond = True

class Tests(unittest.TestCase):
    @unittest.skip('This test never runs')
    def test_no_run(self):
        print('This will never execute in a test suite.')
        
    # We only run this test if our condition is False
    @unittest.skipIf(cond == True, 'Condition is true')
    def test_when_condition_is_false(self):
        pass
    
    # We only run this test if our condition is True
    @unittest.skipUnless(cond == True, 'Condition is false')
    def test_when_condition_is_true(self):
        pass

FAILURES: 0 ERRORS: 0 SKIPPED: 2 TOTAL: 3
s: test_no_run (__main__.Tests)
s: test_when_condition_is_false (__main__.Tests)


# Skipping tests from fixtures with `SkipTest`
* Throw the `SkipTest` exception from `setUp()` to skip tests

In [12]:
%%unittest_run

class Tests(unittest.TestCase):
    def setUp(self):
        # We skip tests if a necessary resource is not available
        raise unittest.SkipTest(
            'Satellite uplink is not available right now.')
        
    # This test will only be run if the satellite 
    # uplink is available
    def test_contact_satellite(self):
        pass

FAILURES: 0 ERRORS: 0 SKIPPED: 1 TOTAL: 1
s: test_contact_satellite (__main__.Tests)


# Allow failures with `expectedFailure`
*`expectedFailure` allows a failing test to count as passing*

This can be useful for "reminder" tests or work-in-progress. For example, it can be used to alert you when a broken upstream dependency has started working.

The command-line runner will let you know when there are expected failures *and* when there are unexpected successes.

In [15]:
%%unittest_run

class Tests(unittest.TestCase):
    # The failing assertion in this test is not counted as a failure
    @unittest.expectedFailure
    def test_this_will_fail(self):
        self.assertTrue(False)

FAILURES: 0 ERRORS: 0 SKIPPED: 0 EXPECTED FAILURES: 1 TOTAL: 1
ef: test_this_will_fail (__main__.Tests)


# Parameterize tests with `TestCase.subTest`
## Run a test with multiple times
*New in 3.4*

This reduces boilerplate in certain situations since failure of one subtest does not prevent execution of other subtests.

In [16]:
%%unittest_run

import string

class Tests(unittest.TestCase):
    def test_all_ascii(self):
        for word in ['thread', 'tråd', 'island', 'øy']:
            # Each subtest context is treated as an independent test
            with self.subTest(word=word):
                self.assertTrue(all(c in string.ascii_lowercase 
                                    for c in word))

FAILURES: 2 ERRORS: 0 SKIPPED: 0 TOTAL: 1
f: test_all_ascii (__main__.Tests) (word='tråd')
f: test_all_ascii (__main__.Tests) (word='øy')


# A complete example
## This is a good template `unittest`

* Keep tests separate from production code
* Use separate `TestCase` subclasses for different features

![unittest project tree](images/unittest-project-tree.png)

```python
# tests/test_sorting.py

import unittest


class SortingTests(unittest.TestCase):
    def test_basic(self):
        self.assertListEqual(sorted([3, 2, 1]), [1, 2, 3])


if __name__ == '__main__':
    unittest.main()
```